In [226]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

In [227]:
from  sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
df = pd.DataFrame(data['data'] , columns= data['feature_names'])
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


                                                            FEATURE SELECTION

It’s not recommended to perform feature selection before splitting your data into training and test sets. Here’s why:

When you perform feature selection before splitting the data, you’re using information from the entire dataset to select features. This includes information from the test set. This is a form of data leakage, where information from outside the training dataset is used to create the model. This can lead to overly optimistic performance estimates.

To avoid this, you should first split your data into training and test sets, and then perform feature selection using only the training data. This ensures that the feature selection process doesn’t have access to any information from the test set.

## PEARSON CO-EFFICIENT

In [228]:
from sklearn.model_selection import train_test_split

class pearson_coefficient:
    
    def __init__(self , max_features):
        self.max_features = max_features
        
    def pearsons_corr(self, x_train, y_train):
        df = pd.DataFrame(x_train)
        df['target'] = y_train
        correlations = df.corr()['target'].drop('target') #finds the corr of all columns to the target variable 
        
        #using abs coz -> in many cases we are interested in the magnitude (value) instead of direction ( pos / negative)
        sorted_corr = correlations.abs().sort_values(ascending = False) 
        selected_features = sorted_corr.index[: self.max_features].tolist()
        return selected_features[: self.max_features] , sorted_corr[: self.max_features]

# Split your data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, data.target, test_size=0.2, random_state=42)

# Create an instance of the class and call the method with the training data
instance = pearson_coefficient(max_features= 9 )
selected_features , sorted_corr = instance.pearsons_corr(x_train = x_train , y_train = y_train)

print(f"{selected_features}\n\n{sorted_corr}")

['worst concave points', 'mean concave points', 'worst perimeter', 'worst radius', 'mean perimeter', 'worst area', 'mean radius', 'mean area', 'mean concavity']

worst concave points    0.788885
mean concave points     0.778115
worst perimeter         0.774998
worst radius            0.766527
mean perimeter          0.731859
worst area              0.722875
mean radius             0.718073
mean area               0.695171
mean concavity          0.683262
Name: target, dtype: float64


FORWARD SELECTION ( Decision Tree)

This function performs forward feature selection. It starts with no features and iteratively adds the feature that gives the best performance improvement when added to the already selected features. The performance is measured using the accuracy of a decision tree classifier. The function returns the selected features in the order they were added, along with their corresponding scores. The print statement inside the loop provides a step-by-step log of which feature is selected at each step and its corresponding score. This can be helpful for understanding how the feature selection process is progressing. Note that you’ll need to import the necessary libraries (DecisionTreeClassifier and accuracy_score from sklearn) for this code to work. Also, this code assumes that X_train, X_test, y_train, and y_test are defined elsewhere in your code and are passed as arguments to the forward_selection method. These should be your training and testing datasets.

## FORWARD FEATURE SELECTION :

This function performs forward feature selection. It starts with no features and iteratively adds the feature that gives the best performance improvement when added to the already selected features. The performance is measured using the accuracy of a decision tree classifier. The function returns the selected features in the order they were added, along with their corresponding scores. The print statement inside the loop provides a step-by-step log of which feature is selected at each step and its corresponding score. This can be helpful for understanding how the feature selection process is progressing. Note that you’ll need to import the necessary libraries (DecisionTreeClassifier and accuracy_score from sklearn) for this code to work. Also, this code assumes that X_train, X_test, y_train, and y_test are defined elsewhere in your code and are passed as arguments to the forward_selection method. These should be your training and testing datasets.


Empty Set of Features --> {Feature 1} --> {Feature 1, Feature 2} --> {Feature 1, Feature 2, Feature 3} --> ... --> Best Set of Features

>> Sure, let’s consider a simple dataset with three features (F1, F2, F3) and a binary target variable (T).

    > Iteration 1: We start with no features. We then fit the model with each feature (F1, F2, F3) and calculate the accuracy. Let’s say F1 gives the best accuracy. So, we select F1.

    > Iteration 2: We now have F1 in our model. We then try adding each of the remaining features (F2, F3) to F1 and fit the model. Let’s say adding F2 gives the best accuracy. So, we select F2.

    > Iteration 3: We now have F1 and F2 in our model. We then try adding the remaining feature (F3) to F1 and F2 and fit the model. If adding F3 improves the accuracy, we select F3, otherwise, we stop here.

If F3 does not improve the model, our final set of selected features would just be {F1, F2}. This set of features is expected to give us the best predictive performance according to the forward feature selection process.

The goal of forward feature selection is to find the smallest set of features that gives the best predictive performance. The advantage of this method is that it can significantly reduce the dimensionality of the problem, making the model simpler, faster, and less prone to overfitting

The downside is that it can be computationally expensive, especially if the number of features is large, as it requires fitting a model for each subset of features.

In [247]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class FeatureSelector:
    
    #initializing the class with max_features 
    def __init__(self , max_features):
        self.max_features = max_features
        
    #define forward selection method 
    def forward_selection(self , x_train , y_train , x_test , y_test):
        
        #initializing the lists for storing the selected features and scores 
        selected_features = []
        ordered_scores = []
        
        #iterate over the range of maximum features
        for i in range(self.max_features):
            features_left = list(set(x_train.columns) - set(selected_features))
            
            #initialize the best score to 0 and best feature to empty string 
            best_score = 0
            best_feature = ''
            
            #iterate over each feature left to be selected 
            for feature in features_left:
                #create a temporary list of features and append the current feature to it 
                temp_selected_features = selected_features + [feature]
                
                #train a decision tree classifier on the training data with temporary selected features
                classifier = DecisionTreeClassifier().fit(x_train[temp_selected_features], y_train)
                
                #predict the test data and calculcate the accuracy 
                y_pred = classifier.predict(x_test[temp_selected_features])
                score = accuracy_score(y_test , y_pred)
                
                #if current score is better than best score update the best score and best feature
                if score > best_score:
                    best_score = score 
                    best_feature = feature
                    
            #append the best feature and its score to respective lists 
            selected_features.append(best_feature)
            ordered_scores.append(best_score)
            
            print(f"Feature {i+1} : {best_feature} with score {best_score}")
        
        return selected_features , ordered_scores
    
#test train split of the data 
x = df
x_train , x_test , y_train , y_test = train_test_split( df , data.target , test_size= 0.2 , random_state= 42)

fs = FeatureSelector(max_features= 12)
selected_features , ordered_scores = fs.forward_selection(x_train , y_train , x_test , y_test)

# Print the selected features and their scores
print("Selected features:", selected_features)
print("Ordered scores:", ordered_scores)

Feature 1 : worst area with score 0.9210526315789473
Feature 2 : worst fractal dimension with score 0.9824561403508771
Feature 3 : concave points error with score 0.9912280701754386
Feature 4 : smoothness error with score 0.9912280701754386
Feature 5 : worst smoothness with score 1.0
Feature 6 : mean fractal dimension with score 1.0
Feature 7 : symmetry error with score 1.0
Feature 8 : mean smoothness with score 1.0
Feature 9 : worst radius with score 1.0
Feature 10 : fractal dimension error with score 1.0
Feature 11 : mean symmetry with score 1.0
Feature 12 : mean texture with score 1.0
Selected features: ['worst area', 'worst fractal dimension', 'concave points error', 'smoothness error', 'worst smoothness', 'mean fractal dimension', 'symmetry error', 'mean smoothness', 'worst radius', 'fractal dimension error', 'mean symmetry', 'mean texture']
Ordered scores: [0.9210526315789473, 0.9824561403508771, 0.9912280701754386, 0.9912280701754386, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
